In [37]:
from google.colab import files
uploaded = files.upload()

Saving deceptive-opinion.csv to deceptive-opinion (1).csv


In [38]:
import pandas as pd
import string

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [39]:
df = pd.read_csv('deceptive-opinion.csv')
# df

In [40]:
column_name = 'text'
def remove_punctuation(text):
    text=text.lower()
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
df[column_name] = df[column_name].apply(remove_punctuation)
# df[column_name]

In [41]:
df.to_csv('modified_dataset.csv', index=False)
# df

In [42]:
df['length_count']=df['text'].apply(len)
# df

In [43]:
import nltk
nltk.download('punkt')
def calculate_bigram_type(text):
    words = nltk.word_tokenize(text.lower())
    bigrams = nltk.bigrams(words)
    bigram_types = set(bigrams)
    return len(bigram_types)


df['bigram_type']=df['text'].apply(calculate_bigram_type)
# df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
# Relationship words
relationship_words = ['husband', 'wife','children','child','son','daughter','aunt','uncle','nephew','niece','sister','brother','grandfather','grandmother','father','mother']

def calculate_relationship_words(text):
    words = nltk.word_tokenize(text.lower())
    relationship_word_count = sum([1 for word in words if word in relationship_words])
    return relationship_word_count
df['relationship_words'] = df['text'].apply(calculate_relationship_words)
# df

In [45]:
#sentiment analysis already present
#for verb and noun count
nltk.download('averaged_perceptron_tagger')
# noun count
def calculate_noun_count(text):
    words = nltk.word_tokenize(text.lower())
    pos_tags = nltk.pos_tag(words)
    noun_count = sum([1 for (word, pos) in pos_tags if pos.startswith('NN')])
    return noun_count

# verb count
def calculate_verb_count(text):
    words = nltk.word_tokenize(text.lower())
    pos_tags = nltk.pos_tag(words)
    verb_count = sum([1 for (word, pos) in pos_tags if pos.startswith('VB')])
    return verb_count

df['noun_count'] = df['text'].apply(calculate_noun_count)
df['verb_count'] = df['text'].apply(calculate_verb_count)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [46]:
# Define positive and negative corpora for sentiment word count
positive_corpus = set([
    'good', 'great', 'excellent', 'wonderful', 'fantastic', 'amazing',
    'awesome', 'terrific', 'lovely', 'nice', 'pleasurable', 'superb',
    'delightful', 'marvelous', 'super', 'brilliant', 'outstanding',
    'perfect', 'incredible', 'joyful', 'magnificent', 'satisfying',
    'happy', 'fortunate', 'admirable', 'exhilarating', 'splendid',
    'praiseworthy', 'commendable', 'beautiful', 'genius', 'top-notch'
])

negative_corpus = set([
    'bad', 'terrible', 'poor', 'awful', 'horrible', 'disappointing',
    'dreadful', 'inferior', 'unpleasant', 'miserable', 'pathetic',
    'lousy', 'painful', 'distressing', 'nasty', 'unfortunate',
    'unsatisfactory', 'depressing', 'displeasing', 'annoying',
    'disgusting', 'abysmal', 'repugnant', 'ugly', 'disheartening',
    'disastrous', 'shoddy', 'flawed', 'atrocious', 'pitiful',
    'regrettable', 'underwhelming', 'subpar', 'crummy', 'horrendous'
])

def calculate_sentiment_word_count(text):
    words = nltk.word_tokenize(text.lower())
    positive_count = sum([1 for word in words if word in positive_corpus])
    negative_count = sum([1 for word in words if word in negative_corpus])
    return positive_count + negative_count

df['sentiment_word_count'] = df['text'].apply(calculate_sentiment_word_count)
df

,deceptive,hotel,polarity,source,text,length_count,bigram_type,relationship_words,noun_count,verb_count,sentiment_word_count
0,truthful,conrad,positive,TripAdvisor,we stayed for a one night getaway with family ...,554,102,0,30,13,1
1,truthful,hyatt,positive,TripAdvisor,triple a rate with upgrade to view room was le...,275,43,0,14,10,1
2,truthful,hyatt,positive,TripAdvisor,this comes a little late as im finally catchin...,1076,196,1,50,24,3
3,truthful,omni,positive,TripAdvisor,the omni chicago really delivers on all fronts...,690,121,0,45,16,1
4,truthful,hyatt,positive,TripAdvisor,i asked for a high floor away from the elevato...,377,71,0,19,15,1
...,...,...,...,...,...,...,...,...,...,...,...
1595,deceptive,intercontinental,negative,MTurk,problems started when i booked the intercontin...,721,127,0,39,24,1
1596,deceptive,amalfi,negative,MTurk,the amalfi hotel has a beautiful website and i...,653,118,2,25,26,1
1597,deceptive,intercontinental,negative,MTurk,the intercontinental chicago magnificent mile ...,762,136,0,37,23,2
1598,deceptive,palmer,negative,MTurk,the palmer house hilton while it looks good in...,433,83,0,21,19,2


In [47]:
# df = df[['deceptive','polarity','length_count', 'bigram_type', 'relationship_words', 'noun_count', 'verb_count','sentiment_word_count']]

#converting poalrity and deceptive columns to int

# when to use onehot encoder and when label encoder?

# 1.Label Encoding:

# Use label encoding when the categorical variable represents ordinal data, where there is a meaningful order or hierarchy among the categories.
# Label encoding assigns each category a unique integer label, preserving the ordinal relationship between the categories.
# Examples of ordinal data include education levels (e.g., "High School," "Bachelor's Degree," "Master's Degree"), survey ratings (e.g., "Poor," "Average," "Excellent"), or age groups (e.g., "Child," "Teenager," "Adult").
# Label encoding is suitable for algorithms that can understand and utilize the ordinal relationship, such as decision trees or gradient boosting models.

# 2.One-Hot Encoding:

# Use one-hot encoding when the categorical variable represents nominal data, where there is no inherent order or hierarchy among the categories.
# One-hot encoding creates binary columns for each unique category, where each column indicates whether a particular category is present or not for a given sample.
# Examples of nominal data include colors (e.g., "Red," "Green," "Blue"), countries (e.g., "USA," "UK," "Canada"), or product types (e.g., "Electronics," "Clothing," "Books").
# One-hot encoding is suitable for algorithms that treat each category independently and do not assume any ordinal relationship, such as logistic regression, neural networks, or support vector machines.
# Note that one-hot encoding can introduce high-dimensional feature space, so it may not be suitable for datasets with a large number of unique categories.
# label_encoder = LabelEncoder()


# # One-hot encoding for the "deceptive" column
# deceptive_encoded = pd.get_dummies(df['deceptive'], prefix='deceptive')
# df = pd.concat([df, deceptive_encoded], axis=1)
# df = df.drop('deceptive', axis=1)

# # One-hot encoding for the "polarity" column
# polarity_encoded = pd.get_dummies(df['polarity'], prefix='polarity')
# df = pd.concat([df, polarity_encoded], axis=1)
# df = df.drop('polarity', axis=1)
label_encoder = LabelEncoder()
df['polarity'] = label_encoder.fit_transform(df['polarity'])
df['deceptive']= label_encoder.fit_transform(df['deceptive'])
df

# so now polarity positive means 1 and negative means 0

,deceptive,hotel,polarity,source,text,length_count,bigram_type,relationship_words,noun_count,verb_count,sentiment_word_count
0,1,conrad,1,TripAdvisor,we stayed for a one night getaway with family ...,554,102,0,30,13,1
1,1,hyatt,1,TripAdvisor,triple a rate with upgrade to view room was le...,275,43,0,14,10,1
2,1,hyatt,1,TripAdvisor,this comes a little late as im finally catchin...,1076,196,1,50,24,3
3,1,omni,1,TripAdvisor,the omni chicago really delivers on all fronts...,690,121,0,45,16,1
4,1,hyatt,1,TripAdvisor,i asked for a high floor away from the elevato...,377,71,0,19,15,1
...,...,...,...,...,...,...,...,...,...,...,...
1595,0,intercontinental,0,MTurk,problems started when i booked the intercontin...,721,127,0,39,24,1
1596,0,amalfi,0,MTurk,the amalfi hotel has a beautiful website and i...,653,118,2,25,26,1
1597,0,intercontinental,0,MTurk,the intercontinental chicago magnificent mile ...,762,136,0,37,23,2
1598,0,palmer,0,MTurk,the palmer house hilton while it looks good in...,433,83,0,21,19,2


In [48]:
print("\nLogistic Regression")
print("-----------------------\n")
# Split the dataset into features (X) and target variable (y)


columns_to_drop = ['deceptive','text','hotel','source']
X = df.drop(columns_to_drop, axis=1)
y = df['deceptive']



model = LogisticRegression(max_iter=1000)
# ------------------------------------------------------------------------------
print("\n--------- using 10 fold cross validation ---------\n")
scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')

# Calculate the average accuracy score
average_accuracy = scores.mean()

# Calculate the average precision score
average_precision = cross_val_score(model, X, y, cv=10, scoring='precision').mean()

# Calculate the average recall score
average_recall = cross_val_score(model, X, y, cv=10, scoring='recall').mean()

print("Average Accuracy:", average_accuracy)
print("Average Precision:", average_precision)
print("Average Recall:", average_recall)

print("\n--------------------80-20 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
print("\n--------------------75-25 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)

print("\n--------------------85-15 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)


Logistic Regression
-----------------------


--------- using 10 fold cross validation ---------

Average Accuracy: 0.5875
Average Precision: 0.5862336251472545
Average Recall: 0.5974999999999999

--------------------80-20 split--------------------

Accuracy: 0.575
Precision: 0.615942
Recall: 0.505952

--------------------75-25 split--------------------

Accuracy: 0.5825
Precision: 0.569231
Recall: 0.572165

--------------------85-15 split--------------------

Accuracy: 0.6208333333333333
Precision: 0.663793
Recall: 0.596899


In [49]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

columns_to_drop = ['deceptive','text','hotel','source']
X = df.drop(columns_to_drop, axis=1)
y = df['deceptive']

# # Split the dataset into training and testing sets (80-20 split)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model=SVC()

print("\n--------- using 10 fold cross validation ---------\n")
cores = cross_val_score(model, X, y, cv=10, scoring='accuracy')

# Calculate the average accuracy score
average_accuracy = scores.mean()

# Calculate the average precision score
average_precision = cross_val_score(model, X, y, cv=10, scoring='precision').mean()

# Calculate the average recall score
average_recall = cross_val_score(model, X, y, cv=10, scoring='recall').mean()

print("Average Accuracy:", average_accuracy)
print("Average Precision:", average_precision)
print("Average Recall:", average_recall)

print("\n--------------------80-20 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
print("\n--------------------75-25 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)

print("\n--------------------85-15 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)


--------- using 10 fold cross validation ---------

Average Accuracy: 0.5875
Average Precision: 0.5080752609581508
Average Recall: 0.5399999999999999

--------------------80-20 split--------------------

Accuracy: 0.478125
Precision: 0.600000
Recall: 0.017857

--------------------75-25 split--------------------

Accuracy: 0.49
Precision: 0.483333
Recall: 0.747423

--------------------85-15 split--------------------

Accuracy: 0.4666666666666667
Precision: 0.600000
Recall: 0.023256


In [50]:
from sklearn.neighbors import KNeighborsClassifier
columns_to_drop = ['deceptive','text','hotel','source']
X = df.drop(columns_to_drop, axis=1)
y = df['deceptive']
model = KNeighborsClassifier(n_neighbors=5)


print("\n--------- using 10 fold cross validation ---------\n")
cores = cross_val_score(model, X, y, cv=10, scoring='accuracy')

# Calculate the average accuracy score
average_accuracy = scores.mean()

# Calculate the average precision score
average_precision = cross_val_score(model, X, y, cv=10, scoring='precision').mean()

# Calculate the average recall score
average_recall = cross_val_score(model, X, y, cv=10, scoring='recall').mean()

print("Average Accuracy:", average_accuracy)
print("Average Precision:", average_precision)
print("Average Recall:", average_recall)

print("\n--------------------80-20 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
print("\n--------------------75-25 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)

print("\n--------------------85-15 split--------------------\n")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Initialize the Logistic Regression model
# Train the model using the training data
model.fit(X_train, y_train)
# Testing
y_pred = model.predict(X_test)                                                                                                                                                                                                                                                                      

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
print("\n---------------------------------------------------")
print("\nClassification report")
print(classification_report(y_test, y_pred))
print("\nConfusion matrix")
print(confusion_matrix(y_test, y_pred))


--------- using 10 fold cross validation ---------

Average Accuracy: 0.5875
Average Precision: 0.49935272971428757
Average Recall: 0.5050000000000001

--------------------80-20 split--------------------

Accuracy: 0.503125
Precision: 0.531915
Recall: 0.446429

--------------------75-25 split--------------------

Accuracy: 0.5
Precision: 0.484694
Recall: 0.489691

--------------------85-15 split--------------------

Accuracy: 0.5166666666666667
Precision: 0.565657
Recall: 0.434109

---------------------------------------------------

Classification report
              precision    recall  f1-score   support

           0       0.48      0.61      0.54       111
           1       0.57      0.43      0.49       129

    accuracy                           0.52       240
   macro avg       0.52      0.52      0.52       240
weighted avg       0.53      0.52      0.51       240


Confusion matrix
[[68 43]
 [73 56]]


In [51]:
from sklearn.feature_extraction.text import CountVectorizer

X=df['text']
y=df['deceptive']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)

cv=CountVectorizer(ngram_range=(1,2))

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000)

print("\n------------Logistic Regression------------\n")
X_train=cv.fit_transform(X_train)
model.fit(X_train, y_train)


y_pred = model.predict(cv.transform(X_test))

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)

# ------------------------------------------------------------------------------

print("\n--------------------SVM--------------------\n")
svm=SVC()
svm.fit(X_train,y_train)
y_pred = svm.predict(cv.transform(X_test))

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)


# ------------------------------------------------------------------------------

print("\n--------------------KNN--------------------\n")
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(cv.transform(X_test))

# accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
print("\nClassification report")
print(classification_report(y_test, y_pred))
print("Confusion matrix")
print(confusion_matrix(y_test, y_pred))


------------Logistic Regression------------

Accuracy: 0.8875
Precision: 0.894040
Recall: 0.870968

--------------------SVM--------------------

Accuracy: 0.83125
Precision: 0.825806
Recall: 0.825806

--------------------KNN--------------------

Accuracy: 0.5875
Precision: 0.688525
Recall: 0.270968

Classification report
              precision    recall  f1-score   support

           0       0.56      0.88      0.69       165
           1       0.69      0.27      0.39       155

    accuracy                           0.59       320
   macro avg       0.63      0.58      0.54       320
weighted avg       0.62      0.59      0.54       320

Confusion matrix
[[146  19]
 [113  42]]
